## Motivation

During the COVID-19 pandemic, given that the most effective vaccines (Pfizer, Moderna) required a two-doses to be effective, governments around the world came up with different dose scheduling strategies to vaccinate their populations.

The [Canadian policy](https://covid19tracker.ca/vaccinationtracker.html) was to prime most people with a first dose, and once a sufficient portion of the population was primed, only then to rollout the second dose. The aim was to provide partial immunity to as many people as possible, as opposed to providing ‘full’ immunity to a subset of the population. *I call this vaccination strategy A.*

However, the USA and most other countries took the second route, which was to prime all their people with a first dose and follow up with a second dose immediately upon the end of the priming period (21-28 days). *I call this vaccination strategy B.*

At the same time, people around the world were self-isolating, distancing and masking themselves to prevent disease spread through contact.

My goal for the project was to simulate the disease spread given these distinct distribution strategies. I asked 2 questions:

1. Which vaccination strategy is ‘better’ when coupled with distancing behaviours? (As I will see, ‘better’ can mean different things.)
    
2. Do distancing behaviors really slow spread?

## Background

Individuals in a population are mobile. When infected individuals carrying the pathogen move around and come into contact with susceptible individuals who are also mobile, transmission occurs and the disease spreads.

At the population level, this type of spread can be mathematically modelled by a compartmental model characterized by an initial value problem. In 1927, Kermack and McKendrick put forth as a series of ordinary differential equations describing a special case of disease spread.

This special case became a classic compartmental model for the spread of infectious disease, known today as the SIR model, where individuals progress unidirectionally between three states (compartments) – susceptible (S), infectious (I) and removed (R)–depending on their interactions with other individuals.

![In the SIR model individuals move unidirectionally from the S, to the I and to the R compartments.](images/SIR_diagram.png)

When a susceptible individual comes into contact with an infectious individual, and disease transmission occurs between them, the susceptible is re-assigned into the infectious compartment. Now, this person may infect further susceptible individuals they come into contact with. When an infectious person recovers or dies, that person is re-assigned to the removed (recovered) compartment, and conferred immunity against the disease. This person may not get re-infected.

Over time, as the epidemic progresses, the number of susceptible individuals declines, the number of removed individuals increases, and transmission rates decrease. This shifting distribution of the population is responsible for the end of the epidemic.

## My Model—SEIRV

I modified the SIR model as depicted in the chart below. 

In my version, when a susceptible individual contracts the pathogen from an infected individual, they are not immediately infectious themselves. They undergo a short incubation period, during which they are assigned to the ‘exposed’ compartment. *(infected ≠ infectious)*

I also added a two-dose vaccine to the scenario. If a susceptible individual is vaccinated with a first dose (primed), they gain partial immunity to the disease-causing agent. This means that if they come into contact with an infectious individual, the probability of disease transmission will be lower than if they were unvaccinated (susceptible).

If the primed individual receives the second dose of the vaccine, they gain complete immunity and move to the removed (recovered) compartment directly, thereby reducing the number of susceptible individuals in the population.

![My modified model including vaccinations and an incubation period](images/SEIRV_diagram.png "I modified the SIR model to include an incubation period and a two-step vaccination course.")

If a susceptible individual is vaccinated with a first dose (primed), they gain partial immunity to the disease-causing agent. This means that if they come into contact with an infectious individual, the probability of disease transmission will be lower than if they were unvaccinated (susceptible).

If the primed individual receives the second dose of the vaccine, they gain complete immunity and move to the removed (recovered) compartment directly, thereby reducing the number of susceptible individuals in the population.

Obviously, this model is a simplification—in a real epidemic, individuals do not move unidirectionally between compartments, and vaccines are not 100% effective. I've attached a list of assumptions in an appendix.

The labels on the arrows in the diagram represent the rates of movement of individuals from one compartment to the next. Please refer to the appendix for transition rate derivations. Notations are defined in the table below.

| Symbol | Definition | Initialization Range | Notes |
| --- | --- | --- | --- |
| $N$ | the total population | [100000,1000000] | 
| $S(t)$ | the number of susceptible individuals at time t, written simply as S | $S(0) = N-I(0)$ |  |
| $E(t)$ | the number of exposed individuals at time t | $E(0) = 0$ |  |
| $I(t)$ | the number of infectious individuals at time  t | $\in(0) = [0,50]$ |  |
| $R(t)$ | the number of removed individuals at time t | $R(0) = 0$ |  |
| $V_1(t)$ | the number of primed individuals who have not contracted the disease at time t | $V_1(0) = 0$ |  |
| $\tau$  |  the average number of contacts per person per unit time | $\tau \in [5,10]$ |  |
| $\alpha$ | the probability of disease transmission upon contact between an infectious individual and a susceptible individual | $\alpha \in [0.1,0.4] $  | I only consider diseases that are not super infectious/super mellow |
| $\beta = \alpha*\tau$ |  |  |  |
|  $\sigma $ | the rate at which individuals transition from the exposed compartment to the infectious compartment. $\sigma $ is calculated as the inverse of the average incubation period in days. | $\sigma \in [0.2,0.5]$  | The incubation period is randomly selected between 2-5 days |
| $\gamma$ | the recovery rate—the rate at which individuals transition from the infectious compartment to the removed compartment. $\gamma$ is calculated as the average infectious period in days. | $\gamma \in [0.07,0.1]$  | The infective period is randomly selected between 5-15 days |
| $\eta$ | the rate at which individuals receive the first dose of the vaccine | $\eta \in [0.001,0.01]$ |  |
| $\psi$ | the rate at which individuals receive the second dose of the vaccine | $\psi \in [0.001, \eta]$ |  |
| $\alpha_v$ | reduced transmission probability due to partial immunity from partial vaccination | $\alpha_v = 0.8*\alpha$  | I assume that a partially vaccinated individual is 20% less likely to contract the disease upon contact with an infectious individual |
| $\mu= \alpha_v*\tau$ |  |  |  |v*\tau$ |  |  |  |

# Simulation

I simulated four situations—a natural epidemic spread, an epidemic adjusted for preventative measures such as masking, social distancing and quarantines, vaccination strategy A, and vaccination strategy B. I ran 200 replications for each scenario.

As summarized in Table 1, I initialized each simulation with a total population ranging between 100000 and 1 million individuals. The number of infectious individuals at the start of the simulation was randomly chosen between 1 and 50. There are no exposed, recovered or vaccinated people at the start of the epidemic, so $N = I(0) + S(0)$.

I used the system of ODEs defined in the appendix to solve the initial value problem and retrieve the number of individuals in each compartment at time t=1. Then I reset the IVP and solved for t=2. I kept going for all subsequent time periods until the end of the epidemic.

I used Scipy's [solve_ivp() function](https://docs.scipy.org/doc/scipy/reference/generated/scipy.integrate.solve_ivp.html) to solve the IVPs.

## Set up—load required packages

In [1]:
# for modelling:
from scipy.integrate import solve_ivp
import numpy as np
import pandas as pd
from math import floor
# for plotting:
import matplotlib.pyplot as plt
import seaborn as sns
# for hypothesis testing:
import scipy.stats as stats
# for pretty printing
from IPython.display import display_html

## Define Functions

Given a vector *y*, whose elements correspond to the number of individuals in the susceptible, exposed, infectious, recovered, singly vaccinated and doubly vaccinated compartments at time t, define a function that calculates the transition rates between the compartments at time t.

Although doubly vaccinated individuals belong to the removed compartment, also record them separately.

In [28]:
def initialize_spread_params(scenario, alpha_range = [0.1, 0.4], tau_range = [5, 10], sigma_range = [0.2, 0.5], gamma_range = [0.07, 0.1], eta_range = [0.001, 0.01]):
    """
    Initializes epidemic spread parameters based on the provided scenario and ranges.
    """
    assert scenario in ["baseline", "prevention", "vax A", "vax B"], f"{scenario} is not a supported scenario!"
    assert len(alpha_range) == 2, f"{alpha_range} should have length 2"
    assert len(tau_range) == 2, f"{tau_range} should have length 2"
    assert len(sigma_range) == 2, f"{sigma_range} should have length 2"
    assert len(gamma_range) == 2, f"{gamma_range} should have length 2"
    assert len(eta_range) == 2, f"{eta_range} should have length 2"
    
    if scenario == "baseline":
        alpha = np.random.uniform(low = alpha_range[0], high = alpha_range[1])
        tau = np.random.randint(low = tau_range[0], high = tau_range[1])
        beta = alpha*tau
        sigma = np.random.uniform(low = sigma_range[0], high = sigma_range[1])
        gamma = np.random.uniform(low = gamma_range[0], high = gamma_range[1])
        mu = 0
        eta = 0
        psi = 0
    elif scenario == "prevention":
        alpha = np.random.uniform(low = alpha_range[0], high = alpha_range[1])
        tau = floor(0.8*np.random.randint(low = tau_range[0], high = tau_range[1])) # distancing measures reduce avg number of contacts
        beta = alpha*tau
        sigma = np.random.uniform(low = sigma_range[0], high = sigma_range[1])
        gamma = np.random.uniform(low = gamma_range[0], high = gamma_range[1])
        mu = 0
        eta = 0
        psi = 0
    elif scenario == "vax A":
        alpha = np.random.uniform(low = alpha_range[0], high = alpha_range[1])
        alpha_v = 0.8*alpha # first dose reduces transmission probability by 20%
        tau = floor(0.8*np.random.randint(low = tau_range[0], high = tau_range[1])) # distancing measures reduce avg number of contacts
        beta = alpha*tau
        sigma = np.random.uniform(low = sigma_range[0], high = sigma_range[1])
        gamma = np.random.uniform(low = gamma_range[0], high = gamma_range[1])
        mu = alpha_v*tau
        eta = np.random.uniform(low = eta_range[0], high = eta_range[1])
        psi = np.random.uniform(low = eta_range[0], high = eta_range[1]) # rate of second dose rollout is likely different to the rate of first dose rollout
    elif scenario == "vax B":
        alpha = np.random.uniform(low = alpha_range[0], high = alpha_range[1])
        alpha_v = 0.8*alpha # first dose reduces transmission probability by 20%
        tau = floor(0.8*np.random.randint(low = tau_range[0], high = tau_range[1])) # distancing measures reduce avg number of contacts
        beta = alpha*tau
        sigma = np.random.uniform(low = sigma_range[0], high = sigma_range[1])
        gamma = np.random.uniform(low = gamma_range[0], high = gamma_range[1])
        mu = alpha_v*tau
        eta = np.random.uniform(low = eta_range[0], high = eta_range[1])
        psi = eta # first and second dose are rolled out at the same rate

    return beta, sigma, gamma, mu, eta, psi

In [29]:
def get_transition_rates(t, y, beta, sigma, gamma, mu, eta, psi, N):
    """
    Returns a list containing the equations for the transition rates for the SEIRV model.
    Inputs include:
        y = an array containing the number of individuals in each compartment at time t
        N = the total population size
        Greeks: the epidemic parameters 
    """
    S = y[0]; E = y[1]; I = y[2]; R = y[3]; V1 = y[4]; V2 = y[5] # y == vector, contains values of S,E,I,R,V1,V2
    dydt = np.array( [-beta*I*S/N - eta*S, #ds/dt
                     beta*I*S/N - sigma*E + mu*V1*I/N, #de/dt
                     sigma*E - gamma*I, #di/dt
                     gamma*I + psi*V1, #dr/dt
                     eta*S - mu*V1*I/N - psi*V1, #dv1/dt
                     psi*V1] ) #dv2/dt
    return dydt

In [34]:
def simulate_epidemic(scenario: str, max_runs = 200, N_range = [100000, 1000000], 
                      alpha_range = [0.1, 0.4], tau_range = [5, 10], sigma_range = [0.2, 0.5], gamma_range = [0.07, 0.1], eta_range = [0.001,0.01], 
                      dose2threshold = 0.7, priming_period = 20):
    """
    Simulates an epidemic spread based on the scenario and spread parameters specified. 
    Repeats the simulation max_runs times with different spread parameters for each run.
    """
    # create containers to store run duration and infection attack rate
    durations = []
    IARs = []
    # initialize run counter
    run = 1

    # begin loop
    while run <= max_runs:
        # initialize the total population
        N = np.random.randint(low = N_range[0], high = N_range[1])
        # initialize the number of individuals in each compartment [S, E, I, R, V1, V2] -- these are stored in y: list
        I = np.random.randint(1,50)
        y = [N-I, 0, I, 0, 0, 0]
        S = y[0]; E = y[1]; I = y[2]; R = y[3]; V1 = y[4]; V2 = y[5]
        # initialize spread parameters
        beta, sigma, gamma, mu_reserve, eta_reserve, psi_reserve = initialize_spread_params(scenario, 
                                                                                            alpha_range, tau_range, sigma_range, gamma_range, eta_range)
        
        # initialize variable to keep track of the infection attack rate
        attack = I/N
        # initialize time
        t = 0
        # begin simulation
        while (I+E) >= 1: # epidemic terminates when number of infectious + exposed is less than 1
            
            # start delivering vaccinations once the number of infected people represents 1% of the population
            if attack < 0.01:
                mu = 0
                eta = 0
                psi = 0
                vax_start_time = t + 1
            else:
                mu = mu_reserve
                eta = eta_reserve
                if scenario == "vax A":
                    if (V1+I+R)/N >= dose2threshold:
                        psi = psi_reserve
                elif scenario == "vax B":
                    if t > vax_start_time + priming_period:
                        psi = psi_reserve
                else:
                    psi = 0
            
            # solve IVP
            soln = solve_ivp(get_transition_rates, (t,t+1), y, t_eval = [t+1], args = (beta, sigma, gamma, mu, eta, psi, N))
            y_new = soln.y.reshape((6,))

            # update infection attack rate
            if y_new[2] > y[2]:
                attack += (y_new[2] - y[2])/N

            # update initial condition
            y = y_new

            # update E and I for terminating condition
            E = y[1]; I = y[2]
            # update R and V1 for the vaccine rollout condition
            R = y[3]; V1 = y[4]

            # move to next day
            t = soln.t[0]

        # store outputs
        IARs.append(attack)
        durations.append(t+1)
        
        run += 1
    return durations, IARs

## Run Simulations

In [35]:
np.random.seed(0)
durations_baseline, IARs_baseline = simulate_epidemic(scenario="baseline")
np.random.seed(0)
durations_prevention, IARs_prevention = simulate_epidemic(scenario="prevention")
np.random.seed(0)
durations_vaxA, IARs_vaxA = simulate_epidemic(scenario="vax A")
np.random.seed(0)
durations_vaxB, IARs_vaxB = simulate_epidemic(scenario="vax B")

In [36]:
results = pd.DataFrame({"durations_baseline": durations_baseline, "IARs_baseline": IARs_baseline,
             "durations_prevention": durations_prevention, "IARs_prevention": IARs_prevention,
             "durations_vaxA": durations_vaxA, "IARs_vaxA": IARs_vaxA,
             "durations_vaxB": durations_vaxB, "IARs_vaxB": IARs_vaxB})

results.head()

,durations_baseline,IARs_baseline,durations_prevention,IARs_prevention,durations_vaxA,IARs_vaxA,durations_vaxB,IARs_vaxB
0,166,0.627148,169,0.608127,169,0.607525,169,0.607525
1,188,0.573632,192,0.537067,184,0.501731,187,0.460104
2,208,0.646972,215,0.605476,169,0.588235,223,0.543684
3,166,0.610230,169,0.589101,168,0.529692,160,0.556653
4,167,0.586537,170,0.567800,198,0.641665,172,0.464109


In [37]:
results.to_csv("results.csv", index=False)

The most effective vaccine distribution strategy may be one that can curb the epidemic the fastest, or one that can minimize the total number of infections. Since we have collected the durations and infection attack rates for each run, we can compare these to determine if there are truly any differences in the 4 cases we simulated. Head over to Hypothesis Testing.Rmd for this analysis!

## Appendix—Assumptions

1. Since I are simulating a short-term epidemic (which I expect to come to an end, rather than become endemic), I ignore vital dynamics of the population (e.g. natural birth and death rates). The total population size is constant.
    
    $$
    N=S(t)+E(t)+I(t)+R(t)
    $$
    
2. By the Cauchy-Lipschiz Theorem, this initial value problem ensures that the system of ODEs has a unique solution.
3. Individuals move freely, and each individual’s ‘actions’ are randomly assigned. 
4. The population is assumed to be wholly mixed.
5. Some individuals in the population become infected, and begin infecting the susceptible individuals they come into contact with. 
6. Newly infected individuals take a few days to become infectious themselves.
7. Once an individual becomes infectious, they remain infectious for a few days.
8. When a person recovers or dies of the disease, that person is moved to the removed compartment, i.e., they cannot contract the disease again or spread it further.
9. The number of susceptible individuals who get infected depends on
    - the number of infectious individuals in the system,
    - the contact rate between infected and susceptible individuals, and
    - the probability of disease transmission from an infected to a susceptible individual.

## Appendix—Transition Rate Derivations



### Transition Rate—Compartment S

The maximum number of susceptibles each infectious individual can infect is the number of susceptibles it comes into contact with. By assumption 4, this can be calculated as $\tau S/N$.

We multiply this by the number of infectious individuals and the transmission probability to determine the number of susceptibles who get infected at any particular moment. This simplifies to $\alpha\tau SI/N = \beta SI/N$.

Susceptible individuals are vaccinated at rate $\eta$.

This means that the total outward flow from compartment S is: $dS/dt = -\beta SI/N - \eta S$

### Transition Rate—Compartment E

The exposed compartment receives individuals from 

- the susceptible compartment at rate $\beta SI/N$
- the primed compartment at a lower rate $\mu V_1 I/N$
- the infectious compartment at a negative rate $-\sigma E$

Overall, $dE/dt = \beta SI/N - \sigma E + \mu V_1 I/N$

### Transition Rate—Compartment I

The infectious compartment gains individuals from the exposed compartment at rate $\sigma E$ and loses them to the removed compartment at rate $\gamma I$.

Therefore, $dI/dt = \sigma E - \gamma I$

### Transition Rate—Compartment V

Susceptible individuals are vaccinated at rate $\eta$. Primed individuals may

- get infected at rate $\mu V_1 I/N$
- receive the second dose at rate $\psi V_1$

So, $dV/dt = \eta S - \psi V_1 - \mu V_1 I/N$

### Transition Rate—Compartment R

The removed compartment receives individuals from 

- the infectious compartment (recovered/dead individuals) at rate $\gamma I$
- the primed compartment (fully vaccinated individuals) at rate $\psi V_1$

The overall transition rate for compartment R is $dR/dt = \gamma I + \psi V_1$

**Note**

💡 In vaccination strategy A, the rate at which the first dose is distributed is not necessarily equal to the rate at which the second dose is distributed.

💡 In vaccination strategy B, after a standard delay, all those who have received the first dose must receive the second dose as well. This necessitates that the rates of first and second inoculation are equal, but the distribution of the second dose begins a few days after the disribution of the first dose.

$$
\eta = \psi \in (0,1)
$$

# Acknowledgement
Thank you Dr Jay Gopalakrishnan of Portland State University for an excellent [example simulation of the SEIR model](https://web.pdx.edu/~gjay/teaching/mth271_2020/html/09_SEIR_model.html)! 

In [38]:
!pip freeze > requirements.txt